In [75]:
import psycopg2
import spacy
import pandas as pd
from spacy.tokens import DocBin

In [76]:
conn = psycopg2.connect(host = "172.16.34.1", port = "5432", user = "mimic_demo", password = "mimic_demo", database = "mimic")
cur = conn.cursor()
cur.execute('SET search_path to mimciii')

Preprocessing the data. Retrieve the social history from mimiciii database and convert it to spaCy format file.

In [79]:
import re
nlp = spacy.load('en_core_web_sm')
doc_bin = DocBin()
# Define a regular expression pattern to match the social history section
#soc_hist_pattern = "social history:\s*(.*?)(?:\.|\n\n)"
soc_hist_pattern = "social history:\s*(.*?)(?=\n{2,}|$)"
#soc_hist_pattern = "social history:\s*\n*\s*(.*?)(?:\.|\n\n)" #work for rowid12
#soc_hist_pattern = "social history:(.+?)\\\\n\\\\n"
#soc_hist_pattern = r"social\s+\s*:.*?(?=\n\n|\n\s*\w+\s*:|$)"

train_data = pd.read_csv('train.csv')
annotation_dict = {}
for index, row in train_data.iterrows():
    row_id = row['row_id']
    start = row['start']
    end = row['end']
    sbdh = row['sbdh']
    if row_id not in annotation_dict:
        annotation_dict[row_id] = []
    annotation_dict[row_id].append({'start': start, 'end': end, 'sbdh': sbdh})
# Iterate through each row_id in the annotation dictionary
for row_id, annotations in annotation_dict.items():

    query = f"""SELECT text FROM mimiciii.noteevents WHERE row_id = {row_id};"""
    cur.execute(query)
    text = cur.fetchone()[0]

    # Process the text using the spaCy pipeline
    doc = nlp(str(text))
    match = re.search(soc_hist_pattern, text,re.IGNORECASE | re.DOTALL)
    soc_hist_text = match.group(1).strip() 
    start_char, end_char = match.span(1)
    doc_soc = nlp(soc_hist_text)
    ents = []
    for annotation in annotations:
        start = annotation['start']
        end = annotation['end']
        sbdh = annotation['sbdh']
        token_selected = None

        token_start_pos = start - start_char
        token_end_pos = end - start_char
        token_selected = soc_hist_text[token_start_pos:token_end_pos]
        if token_selected:
            span = doc_soc.char_span(token_start_pos, token_end_pos, sbdh)
            if span is not None:
                ents.append(span)
                
    indices = set()
    non_overlapping_ents = []
    for ent in ents:
        s, e = ent.start_char, ent.end_char
        if s not in indices and e not in indices:
            non_overlapping_ents.append(ent)
            indices.update(range(s, e))

    doc_soc.ents = non_overlapping_ents        
    doc_bin.add(doc_soc)
doc_bin.to_disk("./data.spacy")  
cur.close()
conn.close()


In [95]:
import random
doc_list = list(doc_bin.get_docs(nlp.vocab))

random.shuffle(doc_list)
train_size = int(len(doc_list) * 0.8)

train_docs = []
test_docs = []
for doc in doc_list[:train_size]:
    train_docs.append(doc)
for doc in doc_list[train_size:]:
    test_docs.append(doc)

train_bin = DocBin(docs=train_docs)
test_bin = DocBin(docs=test_docs)

train_bin.to_disk("train.spacy")
test_bin.to_disk("test.spacy")